<a href="https://colab.research.google.com/github/rutuja-patil24/CMPE-258-Deep_Learning/blob/main/Assignment_08/LangGraph%20Agent%20Design%20Patterns/Pattern_4_Orchestrator_Worker_Pattern_(LangGraph).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Step 1: Imports

In [1]:
!pip install -q langgraph langchain-openai openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.9/148.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 15.0 MB/s eta 0:00:00


In [2]:
!pip install -q langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00


In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, END
from typing import TypedDict
import os
import getpass

#  Prompt user to enter keys securely (not visible in UI)
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
os.environ["LANGCHAIN_PROJECT"] = "orchestrator-worker"
os.environ["LANGCHAIN_TRACING_V2"] = "true"



Enter your OpenAI API key: ··········
Enter your LangSmith API key: ··········


##Step 2: Define State

In [4]:
class EditState(TypedDict):
    submission: str
    feedback: str
    final_version: str

##Step 3: Initialize LLM

In [5]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)

<ipython-input-5-59122b76da0b>:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)


##Step 4: Orchestrator Node

In [7]:
def review_submission(state: EditState) -> dict:
    prompt = ChatPromptTemplate.from_template(
        "You're an editor reviewing this article:\n\n{submission}\n\n"
        "Give feedback on tone, grammar, and structure in 2-3 lines."
    )
    result = (prompt | llm).invoke({"submission": state["submission"]})
    return {"feedback": result.content}


## Step 5: Worker Node

In [8]:
def revise_article(state: EditState) -> dict:
    prompt = ChatPromptTemplate.from_template(
        "Here is the article:\n\n{submission}\n\n"
        "And here's the feedback:\n{feedback}\n\n"
        "Revise the article accordingly and return only the final version."
    )
    result = (prompt | llm).invoke({
        "submission": state["submission"],
        "feedback": state["feedback"]
    })
    return {"final_version": result.content}


##Step 6: Build Workflow

In [9]:
builder = StateGraph(EditState)
builder.add_node("review", review_submission)
builder.add_node("revise", revise_article)
builder.set_entry_point("review")
builder.add_edge("review", "revise")
builder.add_edge("revise", END)
workflow = builder.compile()

##Step 7: Run Workflow

In [10]:
inputs = {
    "submission": "ChatGPT is good tool. It help me write better. I think every people should use it."
}
final_state = workflow.invoke(inputs)

from pprint import pprint
pprint(final_state)


{'feedback': 'The article lacks proper grammar and punctuation, and the tone '
             'is overly informal. It would benefit from more detailed '
             "explanations of how ChatGPT improves writing and why it's "
             'recommended for everyone to use. Consider expanding on the '
             'benefits and features of ChatGPT in a more professional tone.',
 'final_version': 'ChatGPT is a valuable tool that has significantly improved '
                  'my writing skills. Its user-friendly interface and advanced '
                  'capabilities have helped me enhance the quality of my work. '
                  'I believe that ChatGPT is a must-have for anyone looking to '
                  'improve their writing abilities.\n'
                  '\n'
                  'ChatGPT offers a wide range of features that make it an '
                  'essential tool for writers of all levels. From generating '
                  'creative ideas to providing helpful suggestion

##Step 8: Display Outputs

In [11]:
print(" Original Submission:\n", final_state['submission'])
print("\n Feedback from Editor:\n", final_state['feedback'])
print("\n Final Revised Version:\n", final_state['final_version'])

# Optional: Save output to markdown or text file
with open("orchestrator_worker_output.txt", "w") as f:
    f.write(" Original Submission:\n" + final_state['submission'] + "\n\n")
    f.write("Feedback from Editor:\n" + final_state['feedback'] + "\n\n")
    f.write(" Final Revised Version:\n" + final_state['final_version'])


 Original Submission:
 ChatGPT is good tool. It help me write better. I think every people should use it.

 Feedback from Editor:
 The article lacks proper grammar and punctuation, and the tone is overly informal. It would benefit from more detailed explanations of how ChatGPT improves writing and why it's recommended for everyone to use. Consider expanding on the benefits and features of ChatGPT in a more professional tone.

 Final Revised Version:
 ChatGPT is a valuable tool that has significantly improved my writing skills. Its user-friendly interface and advanced capabilities have helped me enhance the quality of my work. I believe that ChatGPT is a must-have for anyone looking to improve their writing abilities.

ChatGPT offers a wide range of features that make it an essential tool for writers of all levels. From generating creative ideas to providing helpful suggestions for sentence structure and grammar, ChatGPT is a versatile platform that can benefit anyone looking to enhance

In [ ]:
from langsmith import traceable, Client

client = Client()

# After your run
runs = client.list_runs(project_name="Orchestrator-Worker", limit=1)
latest_run = next(runs)

print(f"🔗 View Trace: https://smith.langchain.com/public/{latest_run.id}")


🔗 View Trace: https://smith.langchain.com/public/1133a08f-4501-4b3c-8f28-68dad92e0f1b
